In [ ]:
# default_exp core

# module name here

> API details.

In [ ]:
#hide
from nbdev.showdoc import *
import pandas as pd
import logging
import datetime
import sys
import dask.dataframe as dd
import numpy as np
from dask_ml import preprocessing as dask_preprocessing

In [ ]:
!mkdir -p tmp

In [ ]:
#export
def test_eq(a,b): assert a==b, f'{a}, {b}'
    
from collections.abc import Sequence 
def _seq_but_not_str(obj):
    return isinstance(obj, Sequence) and not isinstance(obj, (str, bytes, bytearray))

def listify(obj):
    if _seq_but_not_str(obj):
        return obj

    return [obj]
    
def test_in(items, target):
    items = listify(items)
    missing = [item for item in items if item not in target]
    assert len(missing) == 0, f'{missing} are not in {target}'

In [ ]:
test_in('a', ['a', 'b', 'c'])
test_in(['b', 'c'], ['a', 'b', 'c'])

In [ ]:
#export
def configure_logging(log_dir, log_name, log_lvl='DEBUG', con_log_lvl='INFO'):
    log = logging.getLogger('root')
    already_initialized = any(filter(lambda h: isinstance(h, logging.StreamHandler), log.handlers))
    if already_initialized:
        print("Logging already initialized")
        return logging.getLogger('root')

    numeric_level = getattr(logging, log_lvl, None)
    log_format = '%(levelname)5s [%(asctime)s] %(name)s: %(message)s'
    date_format = '%Y-%m-%d %H:%M:%S'
    logging.basicConfig(
        filename=f'{log_dir}/{log_name}_{datetime.datetime.now().strftime("%Y-%m-%d_%H_%M_%S")}.txt',
        level=numeric_level,
        format=log_format,
        datefmt=date_format)
    log = logging.getLogger('root')
    ch = logging.StreamHandler()
    ch.setLevel(getattr(logging, con_log_lvl, None))
    ch.setFormatter(logging.Formatter(log_format, date_format))
    log.addHandler(ch)

    return log

In [ ]:
#export
def setup_dataframe_copy_logging(log, threshold_mb):
    if not '_original_copy' in dir(pd.DataFrame):
        log.debug('Patching up DataFrame.copy')
        pd.DataFrame._original_copy = pd.DataFrame.copy
    else:
        log.debug('Patching up DataFrame.copy :: already done - skipping.')

    def _loud_copy(self, deep=True):
        size_mb = sys.getsizeof(self) / 1024 / 1024
        if size_mb >= threshold_mb:
            log.debug(f'Copying {size_mb:.1f} MiB (deep={deep})')

        return pd.DataFrame._original_copy(self, deep)

    pd.DataFrame.copy = _loud_copy

In [ ]:
log = configure_logging('./tmp', 'test_log', con_log_lvl='DEBUG')
setup_dataframe_copy_logging(log, threshold_mb=50)

DEBUG [2020-07-06 11:33:26] root: Patching up DataFrame.copy


In [ ]:
df = pd.DataFrame({'a':[1,2,3]})
df2 = df.copy()

In [ ]:
#export
n_total_series = 30490
n_days_total = 1913
raw_dir = 'raw'

In [ ]:
#export
def read_series_sample(log, n):
    df = dd.read_csv(
        f'{raw_dir}/sales_train_validation.csv'
    ).sample(frac = n / n_total_series)
    log.debug(f"Read {len(df)} series")
    return df

In [ ]:
sample = read_series_sample(log, 13)
test_eq(13, len(sample))

DEBUG [2020-07-06 11:33:32] root: Read 13 series


In [ ]:
#export
def melt_sales_series(df_sales_train):
    id_columns = [col for col in df_sales_train.columns if 'id' in col]
    sales_columns = [col for col in df_sales_train.columns if 'd_' in col]
    cat_columns = [col for col in id_columns if col != 'id']

    df_sales_train_melt = df_sales_train.melt(
        id_vars=id_columns,
        var_name='day_id',
        value_name='sales'
    )

    df_sales_train_melt['sales'] = df_sales_train_melt['sales'].astype('int16')

    return df_sales_train_melt

In [ ]:
sample_melt = melt_sales_series(sample)

In [ ]:
test_eq(n_days_total * 13, len(sample_melt))

In [ ]:
#export
def extract_day_ids(df_sales_train_melt):
    sales_columns = [f'd_{col}' for col in range(1, n_days_total+1)]
    mapping = {col: int(col.split('_')[1]) for col in sales_columns}
    df_sales_train_melt['day_id'] = df_sales_train_melt['day_id'].map(mapping)

    import datetime
    d_1_date = pd.to_datetime('2011-01-29')
    mapping = {day:(d_1_date + datetime.timedelta(days=day-1)) for day in range(1, n_days_total+1)}
    df_sales_train_melt['day_date'] = df_sales_train_melt['day_id'].map(mapping)

    mapping = {day:str((d_1_date + datetime.timedelta(days=day-1)).date()) for day in range(1, n_days_total+1)}
    # gonna need it for joining with calendars & stuff
    df_sales_train_melt['day_date_str'] = df_sales_train_melt['day_id'].map(mapping)

    df_sales_train_melt['day_id'] = df_sales_train_melt['day_id'].astype('int16')
    df_sales_train_melt['month_id'] = df_sales_train_melt['day_date'].dt.month.astype('uint8')

    return df_sales_train_melt

In [ ]:
sample_melt = extract_day_ids(sample_melt)
sample_melt.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'day_id',
       'sales', 'day_date', 'day_date_str', 'month_id'],
      dtype='object')

In [ ]:
test_eq(n_days_total * 13, len(sample_melt))

In [ ]:
first_row = sample_melt.head(1)
first_row 

,id,item_id,dept_id,cat_id,store_id,state_id,day_id,sales,day_date,day_date_str,month_id
0,HOBBIES_2_117_CA_1_validation,HOBBIES_2_117,HOBBIES_2,HOBBIES,CA_1,CA,1,0,2011-01-29,2011-01-29,1


In [ ]:
test_in(['day_date', 'day_date_str', 'day_id', 'month_id'], first_row.columns)

In [ ]:
test_eq('2011-01-29', first_row.loc[0, 'day_date_str'])
test_eq(1,            first_row.loc[0, 'day_id'])
test_eq(1,            first_row.loc[0, 'month_id'])

In [ ]:
#export
def join_w_calendar(df_sales_train_melt):
    df_calendar = pd.read_csv(f'{raw_dir}/calendar.csv')

    df_calendar_melt = df_calendar.melt(
        id_vars=['date', 'wm_yr_wk', 'weekday', 'wday', 'year', 'd',
                'event_name_1', 'event_name_2', 'event_type_1', 'event_type_2'],
        value_name='snap_flag',
        var_name='state_id',
        value_vars=['snap_CA', 'snap_TX', 'snap_WI']
    )
    df_calendar_melt['snap_flag'] = df_calendar_melt['snap_flag'].astype('uint8')
    df_calendar_melt['state_id'] = df_calendar_melt['state_id'].str.split('_').str[1]

    df_sales_train_melt =  df_sales_train_melt.merge(
        df_calendar_melt[['date', 'state_id', 'wm_yr_wk', 'snap_flag']],
        left_on=['day_date_str', 'state_id'], right_on=['date', 'state_id'],
#  TODO: dask does not seem to support these       validate='many_to_one'
        )

    df_sales_train_melt['wm_yr_wk'] = df_sales_train_melt['wm_yr_wk'].astype('int16')
    return df_sales_train_melt

In [ ]:
sample_melt = join_w_calendar(sample_melt)

In [ ]:
sample_melt.head(1)

,id,item_id,dept_id,cat_id,store_id,state_id,day_id,sales,day_date,day_date_str,month_id,date,wm_yr_wk,snap_flag
0,HOBBIES_2_117_CA_1_validation,HOBBIES_2_117,HOBBIES_2,HOBBIES,CA_1,CA,1,0,2011-01-29,2011-01-29,1,2011-01-29,11101,0


In [ ]:
# TODO: test_not_in ('date') == dup of day_date_str
test_in(['wm_yr_wk', 'snap_flag'], sample_melt.columns)

In [ ]:
#export
def join_w_prices(partition):
    df_prices = pd.read_csv(f'{raw_dir}/sell_prices.csv')
    partition = partition.merge(
        df_prices,
        on=['store_id', 'item_id', 'wm_yr_wk'],
        how='left'
    )
    partition['sell_price'] = partition['sell_price'].astype('float32')
    partition['sales_dollars'] = (partition['sales'] * partition['sell_price']).astype('float32')
    partition = partition.fillna({'sales_dollars': 0}
    # TODO: doesn't seem to be supported by dask, inplace=True
    )
    return partition

In [ ]:
sample_melt = join_w_prices(sample_melt).persist()

DEBUG [2020-07-06 11:34:12] root: Copying 957.5 MiB (deep=True)
DEBUG [2020-07-06 11:34:23] root: Copying 957.5 MiB (deep=True)


In [ ]:
sample_melt.head(1)

,id,item_id,dept_id,cat_id,store_id,state_id,day_id,sales,day_date,day_date_str,month_id,date,wm_yr_wk,snap_flag,sell_price,sales_dollars
0,HOBBIES_2_117_CA_1_validation,HOBBIES_2_117,HOBBIES_2,HOBBIES,CA_1,CA,1,0,2011-01-29,2011-01-29,1,2011-01-29,11101,0,NaN,0.0


In [ ]:
test_in(['sell_price', 'sales_dollars'], sample_melt.columns)

In [ ]:
#export
def to_parquet(sales_series, file_name, processed_dir, LOG):    
    LOG.debug('Setting index')
    sales_series = sales_series.set_index(sales_series['id'])
    LOG.debug('Setting index - done')
    encoders = {}
    # TODO: dask supposedly does this on its own with sensible defaults
    # sales_series['parquet_partition'] = np.random.randint(0, 100, sales_series.shape[0])

    # this one is a dup of day_date_str which is harder to squeeze through the rest of the pipeline (yay petastorm)
    if 'day_date' in sales_series.columns:
        LOG.debug(f"Dropping 'day_date' from {sales_series.columns}")
        sales_series = sales_series.drop(['day_date'], axis=1)

    for col in sales_series.columns:
        if col in encoders:
            LOG.debug(f'Skipping: {col} - already encoded')
            continue

        # petastorm can't read these
        if str(sales_series[col].dtype) == 'uint8':
            sales_series[col] = sales_series[col].astype('int')

        if str(sales_series[col].dtype) in ['category', 'object']:
            LOG.debug(f'Encoding: {col}')            
            enc = dask_preprocessing.LabelEncoder()
            #enc = LabelEncoder()
            sales_series[col] = enc.fit_transform(sales_series[col])
            # TODO: update other transforms too!
            encoders[col] = enc

    for name, enc in encoders.items():
        LOG.debug(f"Saving encoder: {name}")
        np.save(f'{processed_dir}/{name}.npy', enc.classes_)

    # TODO: uint -> int, category/object -> int, day_date -> drop
    parquet_file = f'{processed_dir}/{file_name}'
    LOG.debug(f"Saving to {parquet_file}")
    sales_series.to_parquet(
        parquet_file,
        # writing index blows up dask
        # also below keyword is dask, pandas would be: index=False,
        write_index=False,
#        partition_cols=['parquet_partition']
    )

In [ ]:
to_parquet(sample_melt, 'sample', './tmp', log)

DEBUG [2020-07-06 11:52:53] root: Setting index
DEBUG [2020-07-06 11:52:53] root: Setting index - done
DEBUG [2020-07-06 11:52:53] root: Dropping 'day_date' from Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'day_id',
       'sales', 'day_date', 'day_date_str', 'month_id', 'date', 'wm_yr_wk',
       'snap_flag', 'sell_price', 'sales_dollars'],
      dtype='object')
DEBUG [2020-07-06 11:52:53] root: Encoding: id
DEBUG [2020-07-06 11:52:53] root: Encoding: item_id
DEBUG [2020-07-06 11:52:54] root: Encoding: dept_id
DEBUG [2020-07-06 11:52:56] root: Encoding: cat_id
DEBUG [2020-07-06 11:52:58] root: Encoding: store_id
DEBUG [2020-07-06 11:52:59] root: Encoding: state_id
DEBUG [2020-07-06 11:53:02] root: Encoding: day_date_str
DEBUG [2020-07-06 11:53:04] root: Encoding: date
DEBUG [2020-07-06 11:53:08] root: Saving encoder: id
/usr/local/lib/python3.7/site-packages/dask/array/core.py:1333: FutureWarning: The `numpy.save` function is not implemented by Dask array. You

In [ ]:
# TODO: check if these can be read back well with a sibling func
[f'{col}:{sample_melt[col].dtype}' for col in sample_melt.columns]

['id:object',
 'item_id:object',
 'dept_id:object',
 'cat_id:object',
 'store_id:object',
 'state_id:object',
 'day_id:int16',
 'sales:int16',
 'day_date:datetime64[ns]',
 'day_date_str:object',
 'month_id:uint8',
 'date:object',
 'wm_yr_wk:int16',
 'snap_flag:uint8',
 'sell_price:float32',
 'sales_dollars:float32']

In [ ]:
sample_melt['id'].value_counts().compute()

HOUSEHOLD_2_302_CA_4_validation    1913
HOUSEHOLD_2_001_CA_3_validation    1913
HOUSEHOLD_1_179_TX_1_validation    1913
HOBBIES_2_117_CA_1_validation      1913
FOODS_3_739_CA_3_validation        1913
FOODS_3_431_WI_3_validation        1913
FOODS_3_318_WI_1_validation        1913
FOODS_3_119_TX_2_validation        1913
FOODS_2_317_WI_1_validation        1913
FOODS_2_095_TX_3_validation        1913
FOODS_2_076_CA_1_validation        1913
FOODS_1_053_CA_1_validation        1913
FOODS_1_050_TX_2_validation        1913
Name: id, dtype: int64